In [ ]:
!pip install torch
import torch

In [ ]:
!pip install transformers

### tokenizer

In [ ]:
from transformers import BertTokenizer, BertModel

model_name = 'HooshvareLab/bert-fa-zwnj-base'  # Specify the BERT model variant
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

text = "جز عشق نبود هیچ دم ساز مرا از است نی اول "
tokenizer.tokenize(text)

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['جز', 'عشق', 'نبود', 'هیچ', 'دم', 'ساز', 'مرا', 'از', 'است', 'نی', 'اول']

### word embedding

In [ ]:
encoded_input = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    padding='max_length',
    truncation=True,
    max_length=150,  # Specify the desired maximum length of the sequence
    return_tensors='pt'
  )
input_ids = encoded_input['input_ids']
attention_mask = encoded_input['attention_mask']
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    words_embedding = outputs.last_hidden_state

words_embedding
print(encoded_input)

{'input_ids': tensor([[   2, 2585, 5174, 3130, 2476, 3823, 2109, 2426, 1925, 1933, 2226, 2212,
            3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [ ]:
words_embedding.size()


torch.Size([1, 150, 768])

### sentence embedding

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sentence_embedding = torch.mean(words_embedding, dim=1).to(device)  # Shape: [1, 768]
sentence_embedding = sentence_embedding.squeeze(0)
sentence_embedding

tensor([ 8.5538e-02, -7.5624e-01,  1.9884e-01, -7.9048e-01, -1.6724e+00,
        -1.0927e+00, -3.7951e-01, -5.0552e-01, -6.3537e-01,  1.5239e+00,
        -8.8235e-01,  4.4737e-01, -5.0677e-01, -9.2339e-01, -8.2049e-01,
         3.1418e-03, -1.5347e-01, -5.0761e-01, -1.2381e+00,  1.3580e-01,
         6.6931e-01,  5.8996e-01, -2.7252e-01,  1.2988e-01,  9.2616e-01,
        -1.8670e-02, -6.4517e-01,  7.9654e-01,  3.7266e-01, -1.0061e+00,
         4.6812e-01, -1.1246e+00,  5.9129e-01, -3.7471e-01, -9.0176e-01,
         1.2732e+00, -1.0587e+00, -1.8157e-01,  4.9336e-01, -3.8781e-01,
        -2.2805e-01, -1.3571e+00, -7.5881e-01,  2.6017e-01, -2.0029e-01,
        -7.5294e-01, -5.4623e-01, -1.4347e+00,  5.7493e-01, -1.9324e-01,
         3.2255e-01, -2.8284e+00,  1.7928e-01,  4.6602e-01,  1.3844e-01,
         4.3415e-02, -7.6405e-01,  2.8147e-01,  7.5340e-01, -8.6281e-01,
         6.0929e-01,  1.1155e+00,  3.1969e-01,  4.6072e-03,  1.0217e+00,
         1.3754e+00,  2.6589e-01, -1.5667e+00, -1.5

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("xsum")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

### another way to sentence embedding

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from simpletransformers.language_representation import RepresentationModel


In [ ]:
sentences = [text] #it should always be a list
model = RepresentationModel(
        model_type="bert",
        model_name="HooshvareLab/bert-base-parsbert-uncased",
        use_cuda=True
    )
sentence_vectors = model.encode_sentences(sentences, combine_strategy="mean")

In [ ]:
sentence_vectors.shape


(1, 768)

In [ ]:
sentence_vectors

array([[-3.81954819e-01,  2.30204582e-01,  1.06023574e+00,
         6.54916584e-01,  3.59564692e-01, -2.09273651e-01,
         1.53141189e-02, -2.42552698e-01,  3.65337342e-01,
        -1.01184678e+00, -5.97410619e-01,  4.19231266e-01,
         9.83760357e-02,  7.35838473e-01,  2.23398611e-01,
         4.78130281e-01,  3.36386859e-01,  7.21095026e-01,
        -6.74003921e-03,  7.25566149e-01, -5.64357281e-01,
         4.67706732e-02,  3.73310447e-01, -1.60486922e-01,
         7.74339020e-01, -3.88042033e-01,  4.58177418e-01,
         4.50864404e-01,  6.17796183e-01,  4.38354194e-01,
         6.70366883e-01, -4.94346499e-01, -8.34366977e-01,
         4.87070054e-01,  8.76929164e-01,  1.92624941e-01,
         2.08146286e+00,  3.33457202e-01,  4.88152616e-02,
        -1.51644897e+00,  7.79975891e-01,  7.41187036e-02,
         8.25241625e-01, -1.59048378e-01, -3.33060533e-01,
         7.47097790e-01, -5.96441567e-01, -5.82248811e-03,
        -4.86772209e-01,  4.79444772e-01, -4.60152090e-0